# Experimentación métodos iterativos

In [2]:
import os
import numpy as np
import subprocess as sp

In [3]:
dir = "../data"

## Funciones auxiliares

In [4]:
def generar_sistema_edd(tam, sol, tipo):
    A = np.random.randint(1, 40, (tam, tam))
    suma = np.sum(np.abs(A), axis=1)
    A += np.diag(suma)

    b = A @ sol

    if not os.path.exists(f"{dir}/{tipo}"):
        os.makedirs(f"{dir}/{tipo}")

    np.savetxt(f"{dir}/{tipo}/matriz_edd_{tam}.txt", np.hstack((A, b[:, np.newaxis])), "%d", " ", comments="", header=f"{tam}")

def generar_sistema_simetrico(tam, sol, tipo):
    A = np.random.randint(1, 40, (tam, tam))
    A = A.T @ A

    b = A @ sol

    if not os.path.exists(f"{dir}/{tipo}"):
        os.makedirs(f"{dir}/{tipo}")

    np.savetxt(f"{dir}/{tipo}/matriz_sim_{tam}.txt", np.hstack((A, b[:, np.newaxis])), "%d", " ", comments="", header=f"{tam}")

def generar_sistema_triangular(tam, sol, tipo):
    A = np.tril(np.random.randint(1, 40, (tam, tam)))
    b = A @ sol

    if not os.path.exists(f"{dir}/{tipo}"):
        os.makedirs(f"{dir}/{tipo}")

    np.savetxt(f"{dir}/{tipo}/matriz_triang_{tam}.txt", np.hstack((A, b[:, np.newaxis])), "%d", " ", comments="", header=f"{tam}")

In [5]:
def correr_algoritmo(dataset, tipo, tam, metodo, iteraciones, tol):        
    proceso = sp.run(["../src/iterativo", f"{dir}/{dataset}/matriz_{tipo}_{tam}.txt", metodo, str(iteraciones), str(tol)], capture_output=True, text=True)
    proceso.check_returncode()

    return np.array(proceso.stdout.split(" "), dtype=np.float64)


In [6]:
def crear_archivos(n, low, high, tipo):
    for i in range(2, n + 1):
        tam = 2 ** i
        x = np.random.randint(low, high, tam)
        
        generar_sistema_edd(tam, x, tipo)
        generar_sistema_simetrico(tam, x, tipo)
        generar_sistema_triangular(tam, x, tipo)

        np.savetxt(f"{dir}/{tipo}/vector_x_{tam}.txt", x, "%d", " ")

## Generar instancias

In [7]:
n = 8

### Vector solucion chico

In [8]:
crear_archivos(n, 0, 10, "chico")

### Vector solucion mediano

In [9]:
crear_archivos(n, 50, 100, "mediano")

### Vector solucion grande

In [10]:
crear_archivos(n, 200, 300, "grande")

## Correr algoritmos

In [11]:
# res = []
# correr_algoritmo("chico", "LU", 1000, 1e-20)

-Error de aproximacion entre valor actual y final (por cantidad de iteraciones) para matrices de diferentes tamaños
-Tiempo final de computo por iteraciones y por tamaño de matrices
-LU tiempo de computo por tamaño de matriz. Error numerico (comparar con x real).
-Comparar LU con todos los metodos iterativos.

## Generación de Gráficos

### Error

In [12]:
def cargar_soluciones(tipo):
    for i in range(2, n + 1):
        tam = 2 ** i
        xs.append(np.genfromtxt(f"{dir}/{tipo}/vector_x_{tam}.txt"))

In [15]:
skipped = 0
datasets = ["chico", "mediano", "grande"]
algoritmos = ["J", "JS", "GS", "GSS"]
tipos = ["edd", "sim", "triang"]
max_iter = 500
step = 50
tolerancia = 1e-10
resultados = {}

for ds in datasets:
    for algoritmo in algoritmos:
        resultados[f"{algoritmo}"] = []
        for tipo in tipos:
            for i in range(2, n + 1):
                for j in range(1, max_iter + 1, step):
                    # Aca va el timeit
                    t = %timeit -r 5 -o correr_algoritmo(ds, tipo, 2 ** i, algoritmo, j, tolerancia)
                    print(type(t))
                    res = correr_algoritmo(ds, tipo, 2 ** i, algoritmo, j, tolerancia)
                    resultados[f"{algoritmo}"].append(
                        {
                         "tipo": f"{tipo}",
                         "tamanio": 2**i,
                         "niter": j,
                         "tiempo": t,
                         "res": res
                        })


for ds in datasets:
    resultados["LU"] = []
    for tipo in tipos:
        for i in range (2, n + 1):
            t = %timeit -r 5 -o correr_algoritmo(ds, tipo, 2 ** i, "LU", 0, tolerancia)
            res = correr_algoritmo(ds, tipo, 2 ** i, "LU", 0, tolerancia)
            resultados["LU"].append(
                {
                 "tipo": {tipo},
                 "tamanio": {2**i},
                 "tiempo": t,
                 "res": res
                })
resultados

1.89 ms ± 297 µs per loop (mean ± std. dev. of 5 runs, 100 loops each)
<class 'IPython.core.magics.execution.TimeitResult'>
1.85 ms ± 116 µs per loop (mean ± std. dev. of 5 runs, 1,000 loops each)
<class 'IPython.core.magics.execution.TimeitResult'>
1.88 ms ± 139 µs per loop (mean ± std. dev. of 5 runs, 100 loops each)
<class 'IPython.core.magics.execution.TimeitResult'>


KeyboardInterrupt: 